In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
####################################################
############# Setear segun cada maquina ############
os.chdir("C:/Users/herna/labo3_empresa3_repo/datasets")

In [2]:
#selecciono que archivo procesar segun el tipo de transformacion a tilizar
#//sin_norm=sin normalizacion //cero_uno=min max (0-1) //media_sd=standard media y desvio
tipos_norm = ['sin_norm', 'cero_uno', 'media_sd']

normalizacion = tipos_norm[0] 

#data = "emp3_sellout_base_period_product.csv"
data_entrada = "emp3_sellout_lags_deltalags_product_categorias" #FE que dejo el NB de EMi BASE
data_salida = "emp3_sellout_base_period_product_FE"

if normalizacion == tipos_norm[0]:
    data_entrada = data_entrada+"_"+normalizacion+".csv"
    data_salida = data_salida+"_"+normalizacion+".csv"
    print('File in: '+data_entrada)
    print('File out: '+data_salida)
    
elif normalizacion == tipos_norm[1]:
    data_entrada = data_entrada+"_"+normalizacion+".csv"
    data_salida = data_salida+"_"+normalizacion+".csv"
    print('File in: '+data_entrada)
    print('File out: '+data_salida)
    
elif normalizacion == tipos_norm[2]:
    data_entrada = data_entrada+"_"+normalizacion+".csv"
    data_salida = data_salida+"_"+normalizacion+".csv"
    print('File in: '+data_entrada)
    print('File out: '+data_salida)


File in: emp3_sellout_lags_deltalags_product_categorias_sin_norm.csv
File out: emp3_sellout_base_period_product_FE_sin_norm.csv


In [3]:
#Lea el Dataset base
df = pd.read_csv(data_entrada)

columnas_a_eliminar = ['periodo']
df = df.drop(columnas_a_eliminar, axis=1)
df = df.rename(columns={'periodo_fecha': 'periodo'})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31243 entries, 0 to 31242
Columns: 300 entries, periodo to cat3_delta_lag_35
dtypes: bool(1), float64(292), int64(3), object(4)
memory usage: 71.3+ MB


**setting the parametros generales**

In [4]:
#periods_to_train = ['2019-10-01','2019-09-01','2019-08-01','2019-07-01','2019-06-01','2019-05-01','2019-04-01','2019-03-01','2019-02-01','2019-01-01','2018-12-01','2018-11-01' ]
#periods_to_train = ['2019-10-01']
#de Abril 2019 y 13 meses hacia atras
periods_to_train = ['2019-04-01','2019-03-01','2019-02-01','2019-01-01','2018-12-01','2018-11-01','2018-10-01','2018-09-01','2018-08-01','2018-07-01','2018-06-01','2018-05-01','2018-04-01']

In [5]:
first_df = True
i = 1
for periodo in periods_to_train:
    
    #cada periodo
    print("Procesando periodo:", periodo)
    
    #tomo periodo para generar mi dataset de train
    df_train = df[df['periodo'] == periodo]
    
    #========================================================================================
    #genero el campo tn_2 target
    #========================================================================================
    fecha_2 = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
    fecha_2 += relativedelta(months=2)  # Suma 2 meses a la fecha
    fecha_2 = fecha_2.strftime('%Y-%m-%d')
    print("periodo tn_2:", fecha_2)
    filtro = df['periodo'] == fecha_2
    df_filtrado = df[filtro]

    # Seleccionar solo las columnas 'product_id' y 'tn', y renombrar 'tn' como 'tn_2'
    df_nuevo = df_filtrado[['product_id', 'tn']].rename(columns={'tn': 'tn_mas_2'})
    df_train = pd.merge(df_train, df_nuevo, on='product_id', how='left')
    
    #========================================================================================
    #tomo rango de fechas para 13 lags campo lag tn
    #========================================================================================
    fecha = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
    fecha += relativedelta(months=-13)  # Suma 2 meses a la fecha
    fecha_inicio = fecha.strftime('%Y-%m-%d')
    print("Fecha ini lags:", fecha.strftime('%Y-%m-%d'))
    
    fecha = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
    fecha += relativedelta(months=-0)  # Suma 2 meses a la fecha
    fecha_fin = fecha.strftime('%Y-%m-%d')
    print("Fecha fin lags:", fecha.strftime('%Y-%m-%d'))
    fecha_inicio = pd.Timestamp(fecha_inicio)
    fecha_fin = pd.Timestamp(fecha_fin)

    lag = 13
    # Iterar a través de los periodos en el rango
    for fecha in pd.date_range(start=fecha_inicio, end=fecha_fin, freq='M'):
        #print('entro')
        # Obtiene el primer día del mes para cada fecha
        fecha = pd.to_datetime(fecha)
        primer_dia_del_mes = fecha - pd.DateOffset(days=fecha.day - 1)
        primer_dia_del_mes = primer_dia_del_mes.strftime('%Y-%m-%d')
        #print(primer_dia_del_mes)
        # Filtrar el DataFrame por el periodo actual
        filtro = df['periodo'] == primer_dia_del_mes
        df_filtrado = df[filtro]
        # Seleccionar solo las columnas 'product_id' y 'tn', y renombrar 'tn' como 'lag_tn_xx'
        df_nuevo = df_filtrado[['product_id', 'tn']].rename(columns={'tn': 'lag_tn_'+str(lag)})


        df_train = pd.merge(df_train, df_nuevo, on='product_id', how='left')
        lag = lag - 1

    #========================================================================================
    #tomo rango de fechas para 13 lags campo lag months
    #========================================================================================

    lag = 13
    # Iterar a través de los periodos en el rango
    for fecha in pd.date_range(start=fecha_inicio, end=fecha_fin, freq='M'):
        # Obtiene el primer día del mes para cada fecha
        fecha = pd.to_datetime(fecha)
        primer_dia_del_mes = fecha - pd.DateOffset(days=fecha.day - 1)
        primer_dia_del_mes = primer_dia_del_mes.strftime('%Y-%m-%d')
        #print(primer_dia_del_mes)
        # Filtrar el DataFrame por el periodo actual
        filtro = df['periodo'] == primer_dia_del_mes
        df_filtrado = df[filtro]
        #print(str(fecha.month))
        df_train['lag_month_'+str(lag)] = str(fecha.month)
        lag = lag - 1

    #========================================================================================
    #genero el campo lag_tn_mean_2_3_6_12 
    #========================================================================================
    mean_periods = [-2,-3,-6,-12]
    fecha_from = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
    fecha_from += relativedelta(months=-1)  # Suma 2 meses a la fecha
    fecha_from = fecha_from.strftime('%Y-%m-%d')
    print("mean from:", fecha_from)
    
    for mean_period in mean_periods:
        fecha_to = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
        fecha_to += relativedelta(months=mean_period)  # Suma 2 meses a la fecha
        fecha_to = fecha_to.strftime('%Y-%m-%d')
        print("mean to "+str(mean_period) , fecha_to)
        filtro = (df['periodo'] >= fecha_to) & (df['periodo'] <= fecha_from)
        df_filtrado = df[filtro]
        df_filtrado = df_filtrado.groupby(['product_id']).agg({'tn': 'mean'}).reset_index()
        df_filtrado = df_filtrado.rename(columns={'tn': 'lag_tn_mean_'+str(abs(mean_period))})
        df_train = pd.merge(df_train, df_filtrado, on='product_id', how='left')

    #========================================================================================
    #genero el campo lag_tn_sum_q1_q2_q3_q4 y lag_tn_mean_q1_q2_q3_q4
    #========================================================================================
    quarters = [[-1,-3], [-4,-6], [-7,-9], [-10,-12]]
    quarters_name = ['q1', 'q2', 'q3', 'q4']
    
    for i in range(len(quarters)):
        
        fecha_from = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
        fecha_from += relativedelta(months=quarters[i][0])  # Suma 2 meses a la fecha
        fecha_from = fecha_from.strftime('%Y-%m-%d')
        print("q from:", fecha_from)

        fecha_to = datetime.strptime(periodo, '%Y-%m-%d')  # Convierte la cadena a objeto datetime
        fecha_to += relativedelta(months=quarters[i][1])  # Suma 2 meses a la fecha
        fecha_to = fecha_to.strftime('%Y-%m-%d')
        print("q to:", fecha_to)
        
        filtro = (df['periodo'] >= fecha_to) & (df['periodo'] <= fecha_from)
        df_filtrado = df[filtro]
        #para sum
        df_filtrado_sum = df_filtrado.groupby(['product_id']).agg({'tn': 'sum'}).reset_index()
        df_filtrado_sum = df_filtrado_sum.rename(columns={'tn': 'lag_sum_tn_'+quarters_name[i]})
        df_train = pd.merge(df_train, df_filtrado_sum, on='product_id', how='left')
        #para mean
        df_filtrado_mean = df_filtrado.groupby(['product_id']).agg({'tn': 'mean'}).reset_index()
        df_filtrado_mean = df_filtrado_mean.rename(columns={'tn': 'lag_mean_tn_'+quarters_name[i]})
        df_train = pd.merge(df_train, df_filtrado_mean, on='product_id', how='left')    

        
    if first_df:
        first_df = False
        df_train_final = df_train
    else:
        df_train_final = pd.concat([df_train_final, df_train], axis=0, ignore_index=True)
        

Procesando periodo: 2019-04-01
periodo tn_2: 2019-06-01
Fecha ini lags: 2018-03-01
Fecha fin lags: 2019-04-01
mean from: 2019-03-01
mean to -2 2019-02-01
mean to -3 2019-01-01
mean to -6 2018-10-01
mean to -12 2018-04-01
q from: 2019-03-01
q to: 2019-01-01
q from: 2018-12-01
q to: 2018-10-01
q from: 2018-09-01
q to: 2018-07-01
q from: 2018-06-01
q to: 2018-04-01
Procesando periodo: 2019-03-01
periodo tn_2: 2019-05-01
Fecha ini lags: 2018-02-01
Fecha fin lags: 2019-03-01
mean from: 2019-02-01
mean to -2 2019-01-01
mean to -3 2018-12-01
mean to -6 2018-09-01
mean to -12 2018-03-01
q from: 2019-02-01
q to: 2018-12-01
q from: 2018-11-01
q to: 2018-09-01
q from: 2018-08-01
q to: 2018-06-01
q from: 2018-05-01
q to: 2018-03-01
Procesando periodo: 2019-02-01
periodo tn_2: 2019-04-01
Fecha ini lags: 2018-01-01
Fecha fin lags: 2019-02-01
mean from: 2019-01-01
mean to -2 2018-12-01
mean to -3 2018-11-01
mean to -6 2018-08-01
mean to -12 2018-02-01
q from: 2019-01-01
q to: 2018-11-01
q from: 2018-

In [6]:
#========================================================================================
#genero el campo lag_trend_q1 q2 q3 q4
#========================================================================================

def calculate_trend(row, *lag_column_names):
    # Filtrar valores no nulos para el cálculo de tendencia
    lag_values = [row[col] for col in lag_column_names if not np.isnan(row[col])]

    # Verificar que haya suficiente variación en los datos para un ajuste lineal
    if len(set(lag_values)) > 1:
        coefficients = np.polyfit([1, 2, 3], lag_values, 1)
        return coefficients[0]
    else:
        return 0.0

print("Columns Trend by quarters")
#Q1
column_names = ["lag_tn_1", "lag_tn_2", "lag_tn_3"]
df_train_final[column_names] = df_train_final[column_names].fillna(0)
df_train_final["lag_trend_q1"] = df_train_final.apply(calculate_trend, args=column_names, axis=1).copy()
#Q2
column_names = ["lag_tn_4", "lag_tn_5", "lag_tn_6"]
df_train_final[column_names] = df_train_final[column_names].fillna(0)
df_train_final["lag_trend_q2"] = df_train_final.apply(calculate_trend, args=column_names, axis=1).copy()
#Q3
column_names = ["lag_tn_7", "lag_tn_8", "lag_tn_9"]
df_train_final[column_names] = df_train_final[column_names].fillna(0)
df_train_final["lag_trend_q3"] = df_train_final.apply(calculate_trend, args=column_names, axis=1).copy()
#Q4
column_names = ["lag_tn_10", "lag_tn_11", "lag_tn_12"]
df_train_final[column_names] = df_train_final[column_names].fillna(0)
df_train_final["lag_trend_q4"] = df_train_final.apply(calculate_trend, args=column_names, axis=1).copy()

Columns Trend by quarters


In [7]:
# Mover la columna 'tn_mas_2' target al último lugar
col_T = df_train_final.pop('tn_mas_2')
df_train_final['tn_mas_2'] = col_T

C:\Users\herna\AppData\Local\Temp\ipykernel_5832\250140083.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train_final['tn_mas_2'] = col_T


In [8]:
df_train_final.head()

,periodo,product_id,tn,cust_request_qty,cust_request_tn,plan_precios_cuidados,cat1,cat2,cat3,sku_size,...,lag_mean_tn_q2,lag_sum_tn_q3,lag_mean_tn_q3,lag_sum_tn_q4,lag_mean_tn_q4,lag_trend_q1,lag_trend_q2,lag_trend_q3,lag_trend_q4,tn_mas_2
0,2019-04-01,20001,1647.63848,478,1757.73271,0,HC,ROPA LAVADO,Liquido,3000.0,...,1864.966707,4710.04632,1570.015440,3695.97419,1231.991397,-97.441510,404.255815,15.867770,50.246465,1109.93769
1,2019-04-01,20002,1287.62346,454,1360.44402,0,HC,ROPA LAVADO,Liquido,3000.0,...,1384.918527,3093.52244,1031.174147,3136.42970,1045.476567,91.580995,184.517870,11.583320,-17.309555,928.36431
2,2019-04-01,20003,565.33774,282,569.69482,0,FOODS,ADEREZOS,Mayonesa,475.0,...,1096.696177,2524.53935,841.513117,2210.57046,736.856820,163.364545,271.756710,-128.057280,52.372575,662.38654
3,2019-04-01,20004,466.70901,346,468.21007,0,FOODS,ADEREZOS,Mayonesa,240.0,...,732.527440,2538.70653,846.235510,1700.72775,566.909250,-54.216855,112.053045,-147.720160,81.833810,667.19411
4,2019-04-01,20005,624.99880,327,629.64621,0,FOODS,ADEREZOS,Mayonesa,120.0,...,578.546193,2139.00521,713.001737,1681.15422,560.384740,-62.314745,260.553290,-129.717215,-25.603695,876.39696


In [9]:
# Exportar el DataFrame a un archivo CSV
df_train_final.to_csv(data_salida, index=False)